In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### PAPER
Using the understanding of the papers both proposed by Facebook AI: 
1.     Unsupervised Cross-lingual Representation Learning at Scale (2020) 
         XLM-R, a multilingual masked language model trained on 100 languages
2.     Cross-lingual Language Model Pretraining(2019)
        XLM uses a pre-processing technique (Byte Pairing Encoding) to create a shared a shared vocabulary between two languages.
        The bias is removed from the low-resource langauages by sampling using a multinomial distribution. And a dual-language training mechanism with BERT in order to learn relations between words in different languages.
    
The goal was to "expose the surprising effectiveness of multilingual models over monolingual models, and show
strong improvements on low-resource languages"

### STRUCTURE OF THE TRANSFORMER
Based off the "Attention is all you need" paper. Transfromers are able to handle long range dependencies by focusing on parts of our input sequence while we predicted our output sequence.

1. ENCODER: maps an input sequence of symbol representations (x₁, …, xₙ) to a sequence of representations z = (z₁, …, zₙ). Each encoder has two sub-layers.
    * A multi-head self attention mechanism on the input vectors. Helping the encoder look at others as it encodes a specific word. 
    * A simple, position-wise fully connected feed-forward network that does the post-processing. 
    
    
2. DECODER: Given z, the decoder then generates an output sequence (y₁, …, yₘ) of symbols one element at a time.
    * A masked multi-head self attention mechanism on the output vectors of the previous iteration.
    * A multi-head attention mechanism on the output from encoder and masked multi-headed attention in decoder. (encoder-decoder attention layer that helps the decoder focus on relevant parts of the input sentence)
    * A simple, position-wise fully connected feed-forward network   

### Methods introduced based off the BERT architecture

1. Each training sample consists of a shared vocabulary of the same text in two languages.  In BERT, each sample is built on a mono-lingual dataset. 
2. As in BERT, the goal of the model is to predict the masked tokens. The XLM model predicts the masked word in one language by either attending to the neighbouring words in either of the two lanaguages, encouraging the model to align the representations in both languages. Also, the XLM model can leverage context from one language to predict the masked word in another language, if the expected language is not sufficent for inference. 

The upgraded BERT is denoted as **Translation Language Modeling (TLM)** while the “vanilla” BERT with BPE inputs is denoted as Masked Language Modeling (MLM).

## TPU SETUP

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import gc
import os
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from transformers import *
from sklearn.metrics import *
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F



import warnings
warnings.filterwarnings("ignore")

import os
import gc
import re
import folium
#import textstat
from scipy import stats
from colorama import Fore, Back, Style, init

import math
import numpy as np
import scipy as sp
import pandas as pd

import random
import networkx as nx
from pandas import Timestamp

from PIL import Image
from IPython.display import SVG
#from keras.utils import model_to_dot

import requests
from IPython.display import HTML

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.pyplot as plt

tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import transformers
import tensorflow as tf

from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.optimizers import Adam
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *

from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer,\
                                            CountVectorizer,\
                                            HashingVectorizer

from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer  

import nltk
from textblob import TextBlob

from nltk.corpus import wordnet
from nltk.corpus import stopwords
#from googletrans import Translator
from nltk import WordNetLemmatizer
#from polyglot.detect import Detector
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

stopword=set(STOPWORDS)

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

np.random.seed(0)

In [3]:
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

BATCH_SIZE = 16 * strategy.num_replicas_in_sync

Tensorflow version 2.6.4
REPLICAS:  1
